In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import logging

from data_utils import load_info, create_dataloaders, load_preprocessed_data

logging.basicConfig(filename='model_output.log', level=logging.INFO, format='%(asctime)s - %(message)s')

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.LeakyReLU(negative_slope=0.01))
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [3]:
def l1_regularization(model, l1_lambda):
    l1_norm = sum(p.abs().sum() for p in model.parameters())
    return l1_lambda * l1_norm

def calculate_r2_oos(predictions, actuals):
    numerator = np.sum((actuals - predictions) ** 2)
    denominator = np.sum(actuals ** 2)
    r2_oos = 1 - (numerator / denominator)
    return r2_oos

In [4]:
def train(model, train_loader, valid_loader, criterion, optimizer, epochs, patience, l1_lambda, best_loss, patience_counter):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for data, target in train_loader:
            data = data.float().requires_grad_()
            target = target.float()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output.squeeze(), target) + l1_regularization(model, l1_lambda)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data, target in valid_loader:
                data = data.float()
                target = target.float()
                output = model(data)
                loss = criterion(output.squeeze(), target)
                val_loss += loss.item() * data.size(0)
        
        val_loss /= len(valid_loader.dataset)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')
        
        # Early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    predictions = []
    actuals = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.float()
            targets = targets.float()
            outputs = model(inputs)
            # print outputs by logging
            logging.info(f'outputs: {outputs}')
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)
            predictions.append(outputs.numpy())
            actuals.append(targets.numpy())

    test_loss /= len(test_loader.dataset)
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)

    print(f'Test Loss: {test_loss:.4f}')

    r2_oos = calculate_r2_oos(predictions, actuals)
    print(f'R²_oos: {r2_oos:.4f}')

In [5]:
input_data, target_data = load_preprocessed_data()
print(input_data.shape, target_data.shape)
firm_info, _ = load_info()

train_loader, valid_loader, test_loader, test_index = create_dataloaders(
    input_data, target_data, firm_info,
    train_date='2008-01-01', valid_date='2015-01-01', test_date='2023-11-01', batch_size=2000)

print(len(train_loader), len(valid_loader), len(test_loader))

(576574, 252) (576574, 3)
146 69 72


In [6]:
# Hyperparameters setting
input_dim = input_data.shape[1] - 2
output_dim = 1
learning_rate = 0.001
epochs = 100
patience = 5
l1_lambda = 1e-5

model = NeuralNetwork(input_dim, [64, 32], output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Early stopping
best_loss = float('inf')
patience_counter = 0

In [7]:
train(model, train_loader, valid_loader, criterion, optimizer, epochs, patience, l1_lambda, best_loss, patience_counter)
test(model, test_loader, criterion)

Epoch 1/100, Train Loss: 0.027559, Val Loss: 0.024108
Epoch 2/100, Train Loss: 0.025437, Val Loss: 0.024099
Epoch 3/100, Train Loss: 0.025269, Val Loss: 0.024098
Epoch 4/100, Train Loss: 0.025225, Val Loss: 0.024098
Epoch 5/100, Train Loss: 0.025206, Val Loss: 0.024096
Epoch 6/100, Train Loss: 0.025197, Val Loss: 0.024098
Epoch 7/100, Train Loss: 0.025191, Val Loss: 0.024099
Epoch 8/100, Train Loss: 0.025187, Val Loss: 0.024101
Epoch 9/100, Train Loss: 0.025185, Val Loss: 0.024102
Epoch 10/100, Train Loss: 0.025183, Val Loss: 0.024103
Early stopping triggered
Test Loss: 0.0199
R²_oos: -0.0004
